In [2]:
import os
import pyreadr
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Change working directory to our project folder - REPLACE PATH WITH YOUR OWN DIRECTORY
# wd = 'C:\\Users\\au535365\\Dropbox\\teaching\\css_fall2023'
wd = '/home/rask/Dropbox/teaching/css_fall2023'
os.chdir(wd)

### Prepare Dataframe

In [4]:
# Read in R data
rfile = '/home/rask/Dropbox/research/stigma-paper/data/00-analysis.rds'
df = pyreadr.read_r(rfile)
df = df[None] 

# Make some new columns
red_parties = ['S', 'SF', 'EL', 'ALT', 'RV']
blue_parties = ['V', 'KF', 'LA', 'NB', 'DF']
df['partycolor'] = df['party'].apply(lambda x: 'red' if x in red_parties else 'blue')
df['partyterm'] = df.apply(lambda x: x['partycolor'] + '-' + str(x['period']), axis=1)

# Data cleaning
df['period'] = df['period'].astype(int)
df['period'] = df['period'].astype(str)

parlterms = df.period.unique().tolist()

for term in parlterms:
    df_term = df.loc[df['period'] == term,].reset_index(drop=True)
    df_term.to_csv(f'data/ft-speeches/{term}.csv', index=False)

# Save
# df.to_csv('data/ft-speeches_2000-2021.csv', index=False)

#df.groupby(['partycolor', 'period']).describe()
#df.groupby(['partyterm']).describe()

In [16]:
files = os.listdir('data/ft-speeches')

In [17]:
dfall = pd.DataFrame()
for file in files:
    df_term = pd.read_csv('data/ft-speeches/' + file)
    dfall = pd.concat([dfall, df_term])
dfall.reset_index(drop=True, inplace=True)

### Prepare Fixed Vocab

In [1]:
# # Define function to remove stopwords
# def remove_stop_words(text): 
#     words = text.split() 
  
#     words = [w for w in words if w not in stopwords] 
  
#     return ' '.join(words)

# translator = str.maketrans('', '', punctuation)

# def preprocessing(raw, verbose=True):
    
#     # Preprocesssing of text
#     text_list = []
#     iterations = len(raw)
    
#     for i in range(iterations):
    
#         if i % 10000 == 0 or i == iterations - 1 & verbose:
#             print(f"Iteration {i} ({round(round(i / iterations, 3) * 100, 3)}% completed)", end = "\r")

#         q = samples[i].replace(u'\xa0', u' ')
#         q = re.sub('\d+', '', q)
#         q = q.translate(translator)      
#         q = remove_stop_words(q)
#         q = q.lower()
#         q = q.split()
#         q = [x for x in q if len(x) > 2]

#         text_list.append(q)
#     return text_list

# samples = df['text'].values.tolist()
# text_clean = preprocessing(raw=samples)
# text_full = [' '.join(x) for x in text_clean]

In [ ]:
vectorizer = CountVectorizer(decode_error='ignore', min_df = 50)
X = vectorizer.fit_transform(df['text'])
wordlist = vectorizer.get_feature_names_out()
wordlist = [x for x in wordlist if not re.search(r'[0-9]', x)]
pickle.dump(wordlist, open('data/vocab_min200.pkl', 'wb'))